In [97]:
import pandas as pd
from sqlalchemy import select
from sqlalchemy.sql import join
from models import Player, TeamStats, GameStats, Game
from db_config import get_session, get_database_engine
from manipulation import get_date_n_days_ago, get_todays_date


class Player_Data:
    def __init__(self, player_name, season, start_at):
        self.engine = get_database_engine()
        session = get_session()
        self.player =  session.query(Player).filter(Player.name == player_name).one_or_none()
        self.player_name = player_name
        self.start_at = start_at
        if not self.player:
            raise KeyError(f"No player found with the name {player_name}.")
        
        self.season = season[:5] + season[-2:] if len(season) == 9 else season
        self.team_games = self.get_team_games()
        self.stats = self.get_individual_game_stats()
        session.close()

    def get_team_games(self):
        
        if self.start_at == "Regular Season" or self.start_at == "Playoffs":
            query = (
                select(
                    TeamStats,
                    Game.date.label('game_date'),
                )

                .select_from(
                    join(TeamStats, Game, TeamStats.game_id == Game.id)
                )

                .where(TeamStats.team_id == self.player.team_id)
                .where(Game.season_type == self.start_at)
                )
            return pd.read_sql(query, self.engine)
        
        elif self.start_at.isinstance(int):
            start_date = get_date_n_days_ago(self.start_at)
            query = (
                select(
                    TeamStats,
                    Game.date.label('game_date'),
                )

                .select_from(
                    join(TeamStats, Game, TeamStats.game_id == Game.id)
                )

                .where(TeamStats.team_id == self.player.team_id)
                .where(Game.date <= start_date)
                .where(Game.date < get_todays_date())
                )
            
            return pd.read_sql(query, self.engine)

    def get_individual_game_stats(self):
        if self.start_at == "Regular Season" or self.start_at == "Playoffs":
            query = (
                select(
                    GameStats,
                    Game.date.label('game_date'),
                )

                .select_from(
                    join(GameStats, Game, GameStats.game_id == Game.id)
                )

                .where(GameStats.player_id == self.player.id)
                .where(Game.season_type == self.start_at)
                )
            return pd.read_sql(query, self.engine)
        
        elif self.start_at.isinstance(int):
            start_date = get_date_n_days_ago(self.start_at)
            query = (
                select(
                    GameStats,
                    Game.date.label('game_date'),
                )

                .select_from(
                    join(GameStats, Game, GameStats.game_id == Game.id)
                )

                .where(GameStats.team_id == self.player.player_id)
                .where(Game.date <= start_date)
                .where(Game.date < get_todays_date())
                )
            
            return pd.read_sql(query, self.engine)
        


In [98]:
player_name = "Tyrese Haliburton"
season = "2023-24"
games_start_last_n_or_date_or_season_type = "Playoffs"
player_data = Player_Data(
    player_name,
    season, 
    games_start_last_n_or_date_or_season_type
    )
print(player_data.player.player_id)

1630169


In [99]:
from common_db_functions import get_players_from_db, get_teams_from_db
from excel_funcs import save_as_excel_workbook


season = "2023-24"
season_type = "Playoffs"
players = get_players_from_db()
players_x = [player for player in players if player.name == "Anthony Edwards"]
test_player = players_x[0]
print(test_player.team_id)
players = [player for player in players if player.team_id == test_player.team_id]
player_names = [player.name for player in players]
print(player_names)

for player_name in player_names:
    season = "2023-24"
    games_start_last_n_or_date_or_season_type = "Playoffs"
    player_data = Player_Data(
        player_name,
        season, 
        games_start_last_n_or_date_or_season_type
        )
    team_stats = player_data.team_games
    player_stats = player_data.stats
    filename = f"data_pile/{player_data.player_name}_{player_data.start_at}_{player_data.season}"
    save_as_excel_workbook([team_stats, player_stats], filename)

1610612750
['Jaylen Clark', 'Kyle Anderson', 'Jaden McDaniels', 'Anthony Edwards', 'Jordan McLaughlin', 'Wendell Moore Jr.', 'Josh Minott', 'Nickeil Alexander-Walker', 'Mike Conley', 'Naz Reid', 'Daishen Nix', 'Monte Morris', 'T.J. Warren', 'Rudy Gobert', 'Karl-Anthony Towns', 'Leonard Miller', 'Luka Garza']
